In [1]:
import pandas as pd
from langdetect import detect
import numpy as np
import altair as alt
import spacy
from multiprocessing.dummy import Pool
from tqdm import tqdm
import collections
import dask.dataframe as dd

alt.renderers.enable("jupyterlab")

RendererRegistry.enable('jupyterlab')

In [2]:
file_path = "/Users/teng/UBCO/mds_labs/block4/542/data/DevGPT/cleaned/pr_total.csv"

df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28348 entries, 0 to 28347
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            28348 non-null  int64  
 1   Type                  28348 non-null  object 
 2   URL_pr                28348 non-null  object 
 3   Author                28348 non-null  object 
 4   RepoName              28348 non-null  object 
 5   RepoLanguage          28252 non-null  object 
 6   Number                28348 non-null  int64  
 7   Title_x               28348 non-null  object 
 8   Body                  28150 non-null  object 
 9   CreatedAt             28348 non-null  object 
 10  ClosedAt              27642 non-null  object 
 11  MergedAt              23310 non-null  object 
 12  UpdatedAt             28348 non-null  object 
 13  State                 28348 non-null  object 
 14  Additions             28348 non-null  int64  
 15  Deletions          

In [3]:
# Remove nan answer and promot.
df = df[~df["Answer"].isna()]
df = df[~df["Prompt"].isna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28242 entries, 0 to 28343
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            28242 non-null  int64  
 1   Type                  28242 non-null  object 
 2   URL_pr                28242 non-null  object 
 3   Author                28242 non-null  object 
 4   RepoName              28242 non-null  object 
 5   RepoLanguage          28146 non-null  object 
 6   Number                28242 non-null  int64  
 7   Title_x               28242 non-null  object 
 8   Body                  28052 non-null  object 
 9   CreatedAt             28242 non-null  object 
 10  ClosedAt              27544 non-null  object 
 11  MergedAt              23228 non-null  object 
 12  UpdatedAt             28242 non-null  object 
 13  State                 28242 non-null  object 
 14  Additions             28242 non-null  int64  
 15  Deletions             28

In [4]:
# Filter out Python and English
df = df.loc[df["RepoLanguage"] == "Python"]

# Detect answer language
df["Language"] = df["Answer"].apply(detect)

# Get English only.
df = df.loc[df["Language"] == "en"]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, 7870 to 23117
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            478 non-null    int64  
 1   Type                  478 non-null    object 
 2   URL_pr                478 non-null    object 
 3   Author                478 non-null    object 
 4   RepoName              478 non-null    object 
 5   RepoLanguage          478 non-null    object 
 6   Number                478 non-null    int64  
 7   Title_x               478 non-null    object 
 8   Body                  405 non-null    object 
 9   CreatedAt             478 non-null    object 
 10  ClosedAt              466 non-null    object 
 11  MergedAt              465 non-null    object 
 12  UpdatedAt             478 non-null    object 
 13  State                 478 non-null    object 
 14  Additions             478 non-null    int64  
 15  Deletions             4

In [5]:
df.head()

,Unnamed: 0,Type,URL_pr,Author,RepoName,RepoLanguage,Number,Title_x,Body,CreatedAt,...,MentionedURL,MentionedProperty,MentionedAuthor,MentionedText,MentionedPath,URL_chatgptsharing,Prompt,Answer,ListOfCode,Language
7870,7870,pull request,https://github.com/paul-gauthier/aider/pull/119,joshuavial,paul-gauthier/aider,Python,119,create Dockerfile and scripts for managing it,I don't know if you want this in the codebase ...,2023-07-18T03:46:34Z,...,https://github.com/paul-gauthier/aider/pull/11...,comments.body,joshuavial,This feels like a much nicer approach to me @p...,NaN,https://chat.openai.com/share/4555f0ea-1e7b-49...,How can I setup a github action to automatical...,"Sure, I can certainly help you with that. Here...","[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '...",en
7871,7871,pull request,https://github.com/paul-gauthier/aider/pull/119,joshuavial,paul-gauthier/aider,Python,119,create Dockerfile and scripts for managing it,I don't know if you want this in the codebase ...,2023-07-18T03:46:34Z,...,https://github.com/paul-gauthier/aider/pull/11...,comments.body,joshuavial,This feels like a much nicer approach to me @p...,NaN,https://chat.openai.com/share/4555f0ea-1e7b-49...,How can I setup a github action to automatical...,"Sure, I can certainly help you with that. Here...","[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '...",en
7872,7872,pull request,https://github.com/paul-gauthier/aider/pull/119,joshuavial,paul-gauthier/aider,Python,119,create Dockerfile and scripts for managing it,I don't know if you want this in the codebase ...,2023-07-18T03:46:34Z,...,https://github.com/paul-gauthier/aider/pull/11...,comments.body,joshuavial,This feels like a much nicer approach to me @p...,NaN,https://chat.openai.com/share/4555f0ea-1e7b-49...,How can I setup a github action to automatical...,"Sure, I can certainly help you with that. Here...","[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '...",en
7873,7873,pull request,https://github.com/paul-gauthier/aider/pull/119,joshuavial,paul-gauthier/aider,Python,119,create Dockerfile and scripts for managing it,I don't know if you want this in the codebase ...,2023-07-18T03:46:34Z,...,https://github.com/paul-gauthier/aider/pull/11...,comments.body,joshuavial,This feels like a much nicer approach to me @p...,NaN,https://chat.openai.com/share/4555f0ea-1e7b-49...,How can I setup a github action to automatical...,"Sure, I can certainly help you with that. Here...","[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '...",en
7898,7898,pull request,https://github.com/chitalian/gptask/pull/2,calum-bird,chitalian/gptask,Python,2,Fix: recursive/glob support,"Changes:\r\n`-r` is now a flag, not an argumen...",2023-07-24T18:09:25Z,...,https://github.com/chitalian/gptask/pull/2#iss...,comments.body,chitalian,@calum-bird \r\nhttps://chat.openai.com/share/...,NaN,https://chat.openai.com/share/902cd378-3ebc-4e...,Give me some test commands for this\n\nimport ...,This Python script is a command-line tool that...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '...",en


In [6]:
len(df["URL_pr"].unique())

18

In [7]:
df_url = df.groupby(by=["URL_pr"])

## Interactions

In [8]:
# Mean number of interactions per issuedf_url
ls_url_chatgptsharing = [len(ls) for ls in df_url.aggregate(list)["URL_chatgptsharing"]]
idx_max = np.argmax(ls_url_chatgptsharing)
idx_min = np.argmin(ls_url_chatgptsharing)
print(
    f"Average interactions per pr: {np.mean(ls_url_chatgptsharing)}\n"
    f"Maximum number of interactions: {ls_url_chatgptsharing[idx_max]}\n"
    f"Minimum number of iteractions: {ls_url_chatgptsharing[idx_min]}"
)

Average interactions per pr: 26.555555555555557
Maximum number of interactions: 167
Minimum number of iteractions: 1


In [9]:
df_url_count = df_url["URL_chatgptsharing"].aggregate(len).reset_index()
df_url_count.columns

Index(['URL_pr', 'URL_chatgptsharing'], dtype='object')

In [10]:
df_url_count

,URL_pr,URL_chatgptsharing
0,https://github.com/Email-Generation/email_gene...,167
1,https://github.com/Hochfrequenz/kohlrahbi/pull...,16
2,https://github.com/RND247/Pype-Synthetic-Data-...,32
3,https://github.com/aiplanethub/genai-stack/pul...,1
4,https://github.com/app-sre/qontract-reconcile/...,80
5,https://github.com/bancaditalia/black-it/pull/58,1
6,https://github.com/bbelderbos/htmx-demo/pull/2,48
7,https://github.com/chitalian/gptask/pull/2,16
8,https://github.com/comfyanonymous/ComfyUI/pull...,1
9,https://github.com/jefftriplett/frontmatter-cl...,32


In [11]:
# Checking histogarm
# df_url_count.hist(column="URL_chatgptsharing")
alt.Chart(df_url_count).mark_bar().encode(
    x=alt.X("URL_chatgptsharing:N", title="Number of Interactions with ChatGPT in PR"),
    y="count()",
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


## Average length of Prompt

In [12]:
nlp = spacy.load("en_core_web_sm")

In [13]:
prompts = df["Prompt"].drop_duplicates().to_list()

In [14]:
# Remove punctuation
with Pool() as pool:
    doc = pool.map(nlp, tqdm(prompts, total=len(prompts)))
    

100%|██████████| 200/200 [00:00<00:00, 27979.75it/s]


In [15]:
tokens = []
for d in tqdm(doc):
    token_without_punc = [token for token in d if not token.is_punct]
    tokens.append(token_without_punc)

100%|██████████| 200/200 [00:00<00:00, 2668.62it/s]


In [16]:
# Calculate average length of prompt
ls_len_promot = [len(token) for token in tokens]
idx_max = np.argmax(ls_len_promot)
idx_min = np.argmin(ls_len_promot)
print(
    f"Average length of prompt per pr: {np.mean(ls_len_promot)}\n"
    f"---------------------------\n"
    f"Maximum length of prompt: {prompts[idx_max]}\n"
    f"---------------------------\n"
    f"Minimum length of prompt: {prompts[idx_min]}"
)

Average length of prompt per pr: 64.71
---------------------------
Maximum length of prompt: import re
import requests
from typing import List, Optional, Dict
from dataclasses import dataclass, field

def snake_to_camel(snake_str: str) -> str:
    components = snake_str.split("_")
    return components[0] + "".join(x.title() for x in components[1:])

def to_camel_case(data: dict) -> dict:
    return {snake_to_camel(k): v for k, v in data.items() if v is not None}

def camel_to_snake(camel_str: str) -> str:
    snake_str = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", camel_str)
    return re.sub("([a-z0-9])([A-Z])", r"\1_\2", snake_str).lower()

def to_snake_case(data: dict) -> dict:
    return {camel_to_snake(k): v for k, v in data.items()}

SEARCH_OPTIONS_TYPES = {
    'query': str,
    'num_results': int,
    'include_domains': list,
    'exclude_domains': list,
    'start_crawl_date': str,
    'end_crawl_date': str,
    'start_published_date': str,
    'end_published_date': str,
    'use_au

## Average length of Answer

In [17]:
answers = df["Answer"].drop_duplicates().to_list()


In [18]:
# Remove punctuation
with Pool() as pool:
    doc = pool.map(nlp, tqdm(answers, total=len(answers)))

100%|██████████| 200/200 [00:00<00:00, 133683.00it/s]


In [19]:
tokens = []
for d in tqdm(doc):
    token_without_punc = [token for token in d if not token.is_punct]
    tokens.append(token_without_punc)

100%|██████████| 200/200 [00:00<00:00, 52006.25it/s]


In [20]:
# Calculate average length of prompt
ls_len_answer = [len(token) for token in tokens]
idx_max = np.argmax(ls_len_answer)
idx_min = np.argmin(ls_len_answer)
print(
    f"Average length of answer per pr: {np.mean(ls_len_answer)}\n"
    f"---------------------------\n"
    f"Maximum length of answer: {answers[idx_max]}\n"
    f"---------------------------\n"
    f"Minimum length of answer: {answers[idx_min]}"
)

Average length of answer per pr: 163.705
---------------------------
Maximum length of answer: The error message "mangum.http: An error occurred running the application" is indicating that an error occurred while running your Mangum application within the Lambda environment. This could be due to a variety of reasons, including misconfigured dependencies, incorrect application setup, unhandled exceptions, or issues related to the integration between your code and the Mangum framework.Here are some steps you can take to diagnose and resolve the issue:Check Logs: Review the logs and any stack traces that were printed during the execution of the Lambda function. Look for more detailed error messages or stack traces that can provide insights into the specific error that occurred.Mangum Configuration: Double-check your Mangum application setup and configuration. Ensure that you have properly defined the Mangum app instance and configured it to work with your FastAPI application.Dependencies: